In [1]:
import os

dxf_folder = r"C:\Users\Rafael\Desktop\training_dataset\dxf files\9-characters\individual"
dxf_files = [os.path.join(dxf_folder, file) for file in os.listdir(dxf_folder) if file.endswith('.dxf')]

print(f"Found {len(dxf_files)} DXF files. Processing...")

Found 236 DXF files. Processing...


In [ ]:
from typing import Dict
import time
import win32com.client
from shapely.geometry import box
from collections import defaultdict

acad = win32com.client.GetActiveObject("AutoCAD.Application")
acad.Visible = True # optional: run in background

time.sleep(1) # Wait a moment to allow it to initialize

characterMetrics: Dict[str, Dict[str, Dict]] = defaultdict(dict)
for file in dxf_files:
    
    doc = acad.Documents.Open(file)
    modelSpace = doc.ModelSpace
    
    text = next(entity for entity in modelSpace)
    character = text.TextString
    font = text.StyleName
    min_point, max_point = text.GetBoundingBox()
    bbox = box(min_point[0], min_point[1], max_point[0], max_point[1])
    width = max_point[0] - min_point[0]
    
    characterMetrics[font][character] = {
        "bbox": bbox,
        "height": 1,
        "width": width,
        "insertion": (0, 0)
    }
    
    doc.Close(False)
    time.sleep(0.5)
    

<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject <unknown>>
<COMObject

In [3]:
import pickle

output_file = "character_metrics.pkl"  # or a full path if you want
with open(output_file, "wb") as f: pickle.dump(characterMetrics, f)
print(f"Saved character metrics to {output_file}")

Saved character metrics to character_metrics.pkl


In [4]:
characterMetrics

defaultdict(dict,
            {'Arial': {'d': {'bbox': <POLYGON ((0.777 -0.016, 0.777 1, 0 1, 0 -0.016, 0.777 -0.016))>,
               'height': 1,
               'width': 0.776944065484311,
               'insertion': (0, 0)},
              'e': {'bbox': <POLYGON ((0.777 -0.016, 0.777 0.741, 0 0.741, 0 -0.016, 0.777 -0.016))>,
               'height': 1,
               'width': 0.776944065484311,
               'insertion': (0, 0)},
              'f': {'bbox': <POLYGON ((0.437 0, 0.437 1.017, 0 1.017, 0 0, 0.437 0))>,
               'height': 1,
               'width': 0.43656207366984995,
               'insertion': (0, 0)},
              'g': {'bbox': <POLYGON ((0.777 -0.294, 0.777 0.741, 0 0.741, 0 -0.294, 0.777 -0.294))>,
               'height': 1,
               'width': 0.776944065484311,
               'insertion': (0, 0)},
              'h': {'bbox': <POLYGON ((0.777 0, 0.777 1, 0 1, 0 0, 0.777 0))>,
               'height': 1,
               'width': 0.776944065484311,
    

In [5]:
characterMetrics['Arial']['a']

{'bbox': <POLYGON ((0.777 -0.016, 0.777 0.741, 0 0.741, 0 -0.016, 0.777 -0.016))>,
 'height': 1,
 'width': 0.776944065484311,
 'insertion': (0, 0)}